In [1]:
import cv2
import numpy as np
import glob
import os
import pandas as pd

In [2]:
def process_and_rotate_image(image_path):
    # Load the image
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    inverted_gray = cv2.bitwise_not(gray)

    # Find the contours in the inverted grayscale image
    contours, _ = cv2.findContours(inverted_gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Find the extreme points
    x_largest = y_largest = 0
    x_smallest = y_smallest = float('inf')

    # Iterate through contours to find extreme points
    for contour in contours:
        for point in contour:
            x, y = point[0]
            if x > x_largest:
                x_largest = x
            if x < x_smallest:
                x_smallest = x
            if y > y_largest:
                y_largest = y
            if y < y_smallest:
                y_smallest = y

    # Calculate the center of the box
    center_x = (x_largest + x_smallest) // 2
    center_y = (y_largest + y_smallest) // 2

    # Count the black pixels in each quadrant
    quadrant_counts = {
        "top_right": 0,
        "top_left": 0,
        "bottom_left": 0,
        "bottom_right": 0
    }

    # Define the ranges for each quadrant
    quadrants = {
        "top_right": ((center_x, y_smallest), (x_largest, center_y)),   # Top-right
        "top_left": ((x_smallest, y_smallest), (center_x, center_y)),  # Top-left
        "bottom_left": ((x_smallest, center_y), (center_x, y_largest)),   # Bottom-left
        "bottom_right": ((center_x, center_y), (x_largest, y_largest))     # Bottom-right
    }

    # Count black pixels in each quadrant
    for key, ((start_x, start_y), (end_x, end_y)) in quadrants.items():
        quadrant = inverted_gray[start_y:end_y, start_x:end_x]
        quadrant_counts[key] = cv2.countNonZero(quadrant)

    # Determine the rotation needed
    ordered_quadrants = ['top_right', 'bottom_right', 'bottom_left', 'top_left']
    max_black_pixels_quadrant = max(quadrant_counts, key=quadrant_counts.get)
    rotation_count = ordered_quadrants.index(max_black_pixels_quadrant)

    # Rotate the image counterclockwise
    rotated_image = np.rot90(image, k=rotation_count)

    # Save the rotated image
    rotated_image_path = image_path.replace('.png', '_rotated.png')
    cv2.imwrite(rotated_image_path, rotated_image)

    return rotated_image_path, quadrant_counts, rotation_count

In [3]:
cwd = os.getcwd()
files = glob.glob('*.png')
for file in files:
    rotated_image_path, quadrant_counts, rotation_count = process_and_rotate_image(file)